In [17]:
import cv2
import tensorflow
import tflearn
import numpy as np 
import os
import random
from tqdm import tqdm
from random import shuffle
training_data = []

TRAIN_DIR = '/home/blackperl/Downloads/train'
TEST_DIR = '/home/blackperl/Downloads/test'
IMG_SIZE= 50
LR = 1e-3
MODEL_NAME = 'dogvscats-{}-{}.model'.format (LR, '8conv-basic-video')

In [18]:
def label_img(img):
	word_label = img.split('.')[-3]
	if word_label == 'cat' : return [1,0]
	elif word_label == 'dog': return [0,1]

In [19]:
def create_train_data():
	training_data = []
	for img in tqdm(os.listdir(TRAIN_DIR)):
		label = label_img(img)
		path = os.path.join(TRAIN_DIR, img)
		img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE))
		training_data.append([np.array(img), np.array(label)])
        shuffle(training_data)
	np.save('train_data.npy', training_data)
	return training_data

In [5]:
def process_test_data():
	testing_data = []
	for img in tqdm(os.listdir(TEST_DIR)):
		path = os.path.join(TEST_DIR, img)
		img_num = img.split('.')[0]
		img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE))
		training_data.append([np.array(img), img_num])

	np.save('test_data.npy', testing_data)
	return testing_data

In [6]:
train_data = create_train_data()

100%|██████████| 25000/25000 [04:09<00:00, 100.25it/s]


In [7]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf
tf.reset_default_graph()

convnet = input_data(shape=[None, 50, 50, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)
convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


In [8]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
	model.load(MODEL_NAME)
	print('modle loaded!')
train = train_data[:-500]
test = train_data[-500:]
X= np.array([i[0] for i in train]).reshape(-1, 50, 50, 1)
Y= [i[1] for i in train]

test_x= np.array([i[0] for i in test]).reshape(-1, 50, 50, 1)
test_y= [i[1] for i in test]



In [9]:
model.fit({'input': X}, {'targets': Y}, n_epoch=5, validation_set=({'input': test_x}, {'targets': test_y}), snapshot_step=50, show_metric=True, 
          run_id=MODEL_NAME)
    

Training Step: 1914  | total loss: 0.54154 | time: 113.566s
| Adam | epoch: 005 | loss: 0.54154 - acc: 0.7162 -- iter: 24448/24500
Training Step: 1915  | total loss: 0.53774 | time: 114.848s
| Adam | epoch: 005 | loss: 0.53774 - acc: 0.7180 | val_loss: 0.57023 - val_acc: 0.7080 -- iter: 24500/24500
--


In [10]:
model.save(MODEL_NAME)

INFO:tensorflow:/home/blackperl/catvdog/dogvscats-0.001-8conv-basic-video.model is not in all_model_checkpoint_paths. Manually adding it.


In [15]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
#test_data = process_test_data()
test_data = np.load('test_data.npy')
fig= plt.figure()


for num, data in enumerate(test_data[:12]):
	img_num = data[1]
	img_data= data[0]
	
	
	y = fig.add_subplot(3,4,num+1)
	orig = img_data
	data= img_data.reshape(50,50,1)
	model_out = model.predict([data])[0]
	
	if np.argmax(model_out) == 1: str_label ='Dog'
	else: str_label = 'Cat'	

	y.imshow(orig, cmap='gray')
	plt.title(str_label)
	y.axes.get_xaxis().set_visible(False)
	y.axes.get_yaxis().set_visible(False)
plt.show()

In [22]:
with open('submission-file.csv', 'w') as p:
	p.write('id,label\n')


In [23]:
with open('submission-file.csv','a') as p:
	for data in tqdm(test_data):
		img_num = data[1]
		img_data= data[0]
		orig = img_data
		data= img_data.reshape(50,50,1)
		model_out = model.predict([data])[0]
		p.write('{},{}\n'.format(img_num, model_out[1]))

0it [00:00, ?it/s]
